In [1]:
import pandas as pd
data=pd.read_csv("/content/googleplaystore.csv")
data

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [10]:
# Clean 'Price' column
# The 'Price' column can contain '0', 'Free', or '$X.XX'
# Convert 'Free' and '0' to 0.0, remove '$' and convert to float
data.loc[:, 'Price'] = data['Price'].apply(
    lambda x: float(x.replace('$', '')) if isinstance(x, str) and '$' in x
    else (0.0 if isinstance(x, str) and (x.strip() == '0' or x.strip().lower() == 'free') else x)
)
data.loc[:, 'Price'] = pd.to_numeric(data['Price'], errors='coerce')
data.loc[:, 'Price'] = data['Price'].fillna(0.0) # Fill any NaNs created during conversion with 0.0

# Clean 'Size' column
def clean_size(size):
    if isinstance(size, str):
        size = size.strip()
        if 'M' in size: # Megabytes
            return float(size.replace('M', ''))
        elif 'k' in size: # Kilobytes, convert to Megabytes for consistency
            return float(size.replace('k', '')) / 1024
        elif 'Varies with device' in size:
            return pd.NA
    return pd.NA # Handle unexpected types or values

data.loc[:, 'Size'] = data['Size'].apply(clean_size)
data.loc[:, 'Size'] = data['Size'].fillna(data['Size'].median()) # Fill NaNs with median

# Clean 'Installs' column
# Remove '+' and ',' and convert to numeric
data.loc[:, 'Installs'] = data['Installs'].astype(str).str.replace('+', '', regex=False).str.replace(',', '', regex=False)
data.loc[:, 'Installs'] = pd.to_numeric(data['Installs'], errors='coerce')
data.loc[:, 'Installs'] = data['Installs'].fillna(data['Installs'].median()) # Fill NaNs with median
data.loc[:, 'Installs'] = data['Installs'].astype(int) # Convert to int after filling NaNs

# Now, redefine categorical and numerical columns based on the cleaned data
cat_columns = data.select_dtypes(include='object').columns.tolist()
num_columns = data.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Fill missing values for the remaining columns
for col in cat_columns:
    if data[col].isnull().any(): # Only fill if there are NaNs
        mode = data[col].mode()[0]
        data.loc[:, col] = data[col].fillna(mode)

for col in num_columns:
    if data[col].isnull().any(): # Only fill if there are NaNs
        mean_val = data[col].mean()
        data.loc[:, col] = data[col].fillna(mean_val)

/tmp/ipython-input-1998975151.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.loc[:, 'Price'] = data['Price'].fillna(0.0) # Fill any NaNs created during conversion with 0.0
/tmp/ipython-input-1998975151.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.loc[:, 'Size'] = data['Size'].fillna(data['Size'].median()) # Fill NaNs with median
/tmp/ipython-input-1998975151.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instea

In [3]:
outlier_info = {}

for col in num_columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    outlier_info[col] = outliers.shape[0]

    print(f"{col}: {outliers.shape[0]} outliers")

Rating: 734 outliers


In [4]:
data = data[~((data['Rating'] < lower_bound) | (data['Rating'] > upper_bound))]
print(f"Shape of data after removing outliers: {data.shape}")

Shape of data after removing outliers: (10107, 13)


In [5]:
outlier_info = {}

for col in num_columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    outlier_info[col] = outliers.shape[0]

    print(f"{col}: {outliers.shape[0]} outliers")

Rating: 0 outliers


In [6]:
data['Reviews'] = pd.to_numeric(data['Reviews'], errors='coerce')
print("Converted 'Reviews' to numeric.")

Converted 'Reviews' to numeric.


/tmp/ipython-input-1887813209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Reviews'] = pd.to_numeric(data['Reviews'], errors='coerce')


In [7]:
data.drop_duplicates(inplace=True)
print(f"Number of duplicates after removal: {data.duplicated().sum()}")

Number of duplicates after removal: 0


/tmp/ipython-input-1223857504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop_duplicates(inplace=True)


**Categorical** **Variable** **Handling**
    • One-Hot Encoding
    • Label Encoding
    • Ordinal Encoding
    • Frequency Encoding
    • Target Encoding

**One** **Hot** **Encoding**

In [11]:

data_one_hot_encoding=pd.get_dummies(data,columns=cat_columns)
data_one_hot_encoding=data_one_hot_encoding.astype(int)
data_one_hot_encoding


,Rating,Reviews,"App_""i DT"" Fútbol. Todos Somos Técnicos.",App_+Download 4 Instagram Twitter,App_- Free Comics - Comic Apps,App_.R,App_/u/app,App_058.ba,App_1. FC Köln App,App_10 Best Foods for You,...,Android Ver_5.0 - 7.1.1,Android Ver_5.0 - 8.0,Android Ver_5.0 and up,Android Ver_5.1 and up,Android Ver_6.0 and up,Android Ver_7.0 - 7.1.1,Android Ver_7.0 and up,Android Ver_7.1 and up,Android Ver_8.0 and up,Android Ver_Varies with device
0,4,159,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,967,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,87510,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,215644,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,967,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,4,38,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10837,5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10838,4,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10839,4,114,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


**Label** **Encoding**

In [12]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data_label_encoding=data.copy()
for col in cat_columns:
  data_label_encoding[col]=le.fit_transform(data_label_encoding[col])
data_label_encoding.head(20)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,6435,0,4.1,159,350,9,0,0,1,9,542,111,15
1,2420,0,3.9,967,345,12,0,0,1,12,466,962,15
2,8018,0,4.7,87510,328,14,0,0,1,9,113,431,15
3,7243,0,4.5,215644,356,16,0,0,4,9,798,2625,18
4,6492,0,4.3,967,269,11,0,0,1,11,733,253,20
5,6354,0,4.4,167,297,10,0,0,1,9,870,106,8
6,7289,0,3.8,178,350,10,0,0,1,9,72,253,15
7,4948,0,4.1,36815,360,13,0,0,1,9,703,2269,18
8,4402,0,4.4,13791,364,13,0,0,1,9,1269,1379,10
9,5146,0,4.7,121,272,9,0,0,1,11,647,1354,15


**Ordinal** **Encoding**

In [13]:
from sklearn.preprocessing import OrdinalEncoder
data_ordinal_encoding=data.copy()
oe=OrdinalEncoder()
data_ordinal_encoding['Genres']=oe.fit_transform(data_ordinal_encoding[['Genres']])
data_ordinal_encoding.head(20)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,9.0,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,12.0,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,9.0,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,9.0,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,11.0,"June 20, 2018",1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6,50000,Free,0.0,Everyone,9.0,"March 26, 2017",1.0,2.3 and up
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19.0,50000,Free,0.0,Everyone,9.0,"April 26, 2018",1.1,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29.0,1000000,Free,0.0,Everyone,9.0,"June 14, 2018",6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33.0,1000000,Free,0.0,Everyone,9.0,"September 20, 2017",2.9.2,3.0 and up
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1,10000,Free,0.0,Everyone,11.0,"July 3, 2018",2.8,4.0.3 and up


**Binary** **Encoding**

In [14]:
!pip install category_encoders
import category_encoders as ce
data_binary_encoding = data.copy()
encoder = ce.BinaryEncoder(cols=['Genres'])
data_binary_encoding = encoder.fit_transform(data_binary_encoding)
data_binary_encoding

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 3.1 MB/s eta 0:00:00


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres_0,Genres_1,Genres_2,Genres_3,Genres_4,Genres_5,Genres_6,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.100000,159,19.0,10000,Free,0.0,Everyone,0,0,0,0,0,0,1,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.900000,967,14.0,500000,Free,0.0,Everyone,0,0,0,0,0,1,0,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.700000,87510,8.7,5000000,Free,0.0,Everyone,0,0,0,0,0,0,1,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.500000,215644,25.0,50000000,Free,0.0,Teen,0,0,0,0,0,0,1,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.300000,967,2.8,100000,Free,0.0,Everyone,0,0,0,0,0,1,1,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.500000,38,53.0,5000,Free,0.0,Everyone,0,0,0,1,1,1,0,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.000000,4,3.6,100,Free,0.0,Everyone,0,0,0,1,1,1,0,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,4.193338,3,9.5,1000,Free,0.0,Everyone,1,0,1,0,1,1,0,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.500000,114,13.0,1000,Free,0.0,Mature 17+,0,0,0,0,1,1,1,"January 19, 2015",Varies with device,Varies with device


**Frequency** **Encoding**

In [15]:
data_frequency_encoding = data.copy()
for col in cat_columns:
  data_frequency_encoding[col] = data_frequency_encoding[col].map(data_frequency_encoding[col].value_counts())
data_frequency_encoding

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,1,63,4.100000,159,125,888,8920,8920,7753,56,6,124,1364
1,2,63,3.900000,967,166,479,8920,8920,7753,2,9,48,1364
2,1,63,4.700000,87510,37,661,8920,8920,7753,56,249,9,1364
3,1,63,4.500000,215644,124,271,8920,8920,1099,56,34,1267,362
4,1,63,4.300000,967,57,1014,8920,8920,7753,7,35,241,811
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,1,1814,4.500000,38,34,398,8920,8920,7753,499,5,3,2261
10837,1,1814,5.000000,4,55,668,8920,8920,7753,499,59,732,2261
10838,1,374,4.193338,3,21,765,8920,8920,7753,374,2,732,202
10839,1,219,4.500000,114,1652,765,8920,8920,412,219,1,1267,1186


**Target** **Encoding**

In [16]:
import category_encoders as ce
data_target_encoding = data.copy()

encoder = ce.TargetEncoder(cols=['Category'])

# Fit and transform the 'Category' column using 'Rating' as the target
data_target_encoding['Category'] = encoder.fit_transform(data_target_encoding['Category'], data_target_encoding['Rating'])
data_target_encoding.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.382497,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,4.382497,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.382497,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,4.382497,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,4.382497,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Nominal, low cardinality → One-Hot Encoding

Ordinal features → Ordinal or Label Encoding

High cardinality features → Binary or Target Encoding

Frequency matters → Frequency Encoding

Target-related patterns → Target Encoding


##Train/Test split



Perform a train/test split on the `data_label_encoding` DataFrame, using 'Rating' as the target variable (y) and all other columns as features (X). Set the test size to 20% and a `random_state` for reproducibility. Finally, print the shapes of the resulting `X_train`, `X_test`, `y_train`, and `y_test` sets.

## Select Features and Target

### Subtask:
Choose one of the encoded DataFrames and define your feature matrix (X) and target variable (y).


**Reasoning**:
To prepare the data for modeling, I will define the feature matrix `X` by dropping the target column 'Rating' from the `data_label_encoding` DataFrame, and define the target variable `y` as the 'Rating' column.



In [17]:
X = data_label_encoding.drop('Rating', axis=1)
y = data_label_encoding['Rating']

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (9634, 12)
Shape of y: (9634,)


**Reasoning**:
Now that the features (X) and target (y) have been defined, the next step is to split the data into training and testing sets for model development and evaluation.



In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (7707, 12)
Shape of X_test: (1927, 12)
Shape of y_train: (7707,)
Shape of y_test: (1927,)


## Final Task

### Subtask:
Summarize the train/test split, including the chosen features, target, and the sizes of the training and testing sets.



*   The dataset is now prepared for machine learning model training and evaluation, with 80% dedicated to training and 20% to testing.
*   The next logical step is to train a machine learning model using `X_train` and `y_train`, and then evaluate its performance using `X_test` and `y_test`.
